In [4]:
import pickle
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader
import torch.nn.functional as F
import torchvision.transforms as transforms
import torchvision

# Function to unpack the cifar-10 dataset.
def unpickle(file):
    with open(file, 'rb') as file:
        data = pickle.load(file, encoding='bytes')
    return data[b'data'], data[b'labels']

In [5]:
import os
from torch.utils.data import Dataset
#from utils import unpickle


class LoadTrainingData(Dataset):
    def __init__(self):
        self.trainX = []
        self.trainY = []

        data_dir = './cifar-10/training batches'
        batches = os.listdir(data_dir)

        for batch in batches:
            batch_data, batch_labels = unpickle(os.path.join(data_dir, batch))
            self.trainX.extend(batch_data)
            self.trainY.extend(batch_labels)

    def __getitem__(self, item):
        return self.trainX[item], self.trainY[item]

    def __len__(self):
        return len(self.trainX)

In [6]:
data_dir = './cifar-10/'
b_size=128
transform_train = transforms.Compose([
    transforms.ToTensor(),
    transforms.Normalize((0.4914, 0.4822, 0.4465), (0.2023, 0.1994, 0.2010)),
])

# Normalize the test set same as training set without augmentation
transform_test = transforms.Compose([
    transforms.ToTensor(),
    transforms.Normalize((0.4914, 0.4822, 0.4465), (0.2023, 0.1994, 0.2010)),
])

trainset = torchvision.datasets.CIFAR10(root=data_dir, train=True, download=True, transform=transform_train)
trainloader = torch.utils.data.DataLoader(trainset, batch_size=b_size, shuffle=True, num_workers=2)

testset = torchvision.datasets.CIFAR10(root=data_dir, train=False, download=True, transform=transform_test)
testloader = torch.utils.data.DataLoader(testset, batch_size=1, shuffle=False, num_workers=2)

Files already downloaded and verified
Files already downloaded and verified


In [7]:
class classifier(nn.Module):
    def __init__(self):
        super(classifier, self).__init__()
        self.conv1 = nn.Conv2d(3, 16, kernel_size=3,padding=1,stride=1)
        self.bn1   = nn.BatchNorm2d(16)
        self.conv2 = nn.Conv2d(16, 32, kernel_size=3,padding=1,stride=1)
        self.bn2   = nn.BatchNorm2d(32)
        self.conv3 = nn.Conv2d(32, 64, kernel_size=3,padding=1,stride=1)
        self.bn3   = nn.BatchNorm2d(64)
        self.conv4 = nn.Conv2d(64, 128, kernel_size=3,padding=1,stride=1)
        self.bn4   = nn.BatchNorm2d(128)
        self.conv5 = nn.Conv2d(128, 256, kernel_size=3,padding=1,stride=1)
        self.bn5   = nn.BatchNorm2d(256)
        self.conv6 = nn.Conv2d(256, 512, kernel_size=3,padding=1,stride=1)
        self.bn6   = nn.BatchNorm2d(512)
        
        self.pool = nn.MaxPool2d(kernel_size=2, stride=2)
        self.fc1 = nn.Linear(512, 10)

    def forward(self, x):
        x = F.tanh(self.bn1(self.conv1(x)))
        x = self.pool(x)
        x = F.tanh(self.bn2(self.conv2(x)))
        x = self.pool(x)
        x = F.tanh(self.bn3(self.conv3(x)))
        x = self.pool(x)
        x = F.tanh(self.bn4(self.conv4(x)))
        x = self.pool(x)
        x = F.tanh(self.bn5(self.conv5(x)))
        x = self.pool(x)
        x = F.tanh(self.bn6(self.conv6(x)))

        x = x.view(-1,512)
        x = self.fc1(x)
        return x

In [5]:
cls=classifier()
cls

classifier(
  (conv1): Conv2d(3, 16, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (bn1): BatchNorm2d(16, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (conv2): Conv2d(16, 32, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (bn2): BatchNorm2d(32, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (conv3): Conv2d(32, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (bn3): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (conv4): Conv2d(64, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (bn4): BatchNorm2d(128, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (conv5): Conv2d(128, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (bn5): BatchNorm2d(256, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (conv6): Conv2d(256, 512, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (bn6): BatchNorm2d(512, eps=1e-05, momentum=0.1, affine=True, track_runni

In [8]:
def train_model(model, data, epoch,lr):
    # define the loss function and back propagation algorithm
    criterion = nn.CrossEntropyLoss()
    optimizer = torch.optim.Adam(model.parameters(), lr = lr ,amsgrad=True,weight_decay=5e-4)
    scheduler = optim.lr_scheduler.MultiStepLR(optimizer, milestones=[50,80], gamma=0.1)

    for e in range(epoch):
        for param_group in optimizer.param_groups:
            lr = param_group['lr']
        for i, dataset in enumerate(data):
            inputs, lbl = dataset

            if torch.cuda.is_available():
                inputs, lbl = inputs.cuda(), lbl.cuda()

            # set the gradient for each parameters zero
            optimizer.zero_grad()
            outputs = model(inputs)
            loss = criterion(outputs, lbl)
            loss.backward()
            optimizer.step()
        print('-----[Epoch: %d, loss: %f, Learning Rate: %f]' % (e+1, loss.item(),lr  ))
        scheduler.step()


    print ('Finished Training')


if __name__ == '__main__':
    cls=classifier()
    if torch.cuda.is_available():
        cls.cuda()
    lr = 0.003
    epoch = 50

    train_model(cls, trainloader, epoch,lr)

    # save model
    torch.save(cls.state_dict(), './trained_model.pth')

/home/sandeep/anaconda3/lib/python3.7/site-packages/torch/nn/functional.py:1628: UserWarning: nn.functional.tanh is deprecated. Use torch.tanh instead.
  warnings.warn("nn.functional.tanh is deprecated. Use torch.tanh instead.")


-----[Epoch: 1, loss: 1.284263, Learning Rate: 0.003000]
-----[Epoch: 2, loss: 0.858462, Learning Rate: 0.003000]
-----[Epoch: 3, loss: 0.958564, Learning Rate: 0.003000]
-----[Epoch: 4, loss: 0.687633, Learning Rate: 0.003000]
-----[Epoch: 5, loss: 0.847297, Learning Rate: 0.003000]
-----[Epoch: 6, loss: 0.642916, Learning Rate: 0.003000]
-----[Epoch: 7, loss: 0.537064, Learning Rate: 0.003000]
-----[Epoch: 8, loss: 0.571894, Learning Rate: 0.003000]
-----[Epoch: 9, loss: 0.728765, Learning Rate: 0.003000]
-----[Epoch: 10, loss: 0.549686, Learning Rate: 0.003000]
-----[Epoch: 11, loss: 0.401876, Learning Rate: 0.003000]
-----[Epoch: 12, loss: 0.414722, Learning Rate: 0.003000]
-----[Epoch: 13, loss: 0.494872, Learning Rate: 0.003000]
-----[Epoch: 14, loss: 0.428048, Learning Rate: 0.003000]
-----[Epoch: 15, loss: 0.478827, Learning Rate: 0.003000]
-----[Epoch: 16, loss: 0.515159, Learning Rate: 0.003000]
-----[Epoch: 17, loss: 0.299984, Learning Rate: 0.003000]
-----[Epoch: 18, loss: 

In [9]:
from tqdm import tqdm
import numpy as np

def test():
    trained_model = './trained_model.pth'
    
    cls=classifier()
    cls.load_state_dict(torch.load(trained_model))
    cls.cuda()
    cls.eval()

    classes = ['Airplane', 'Automobile', 'Bird', 'Cat', 'Deer', 'Dog', 'Frog', 'Horse', 'Ship', 'Truck']

    # calculating the accuracy of our classifier;
    print("Calculating accuracy...")
    correct = 0
    total = len(testloader)

    with torch.no_grad():
        for dataset in tqdm(testloader):
            inputs, lbl = dataset

            if torch.cuda.is_available():
                inputs, lbl = inputs.cuda(), lbl.cuda()
            
            out = cls(inputs)
            _, predicted = torch.max(out, 1)

            # calculate the total accuracy
            correct += (predicted == lbl).sum().item()
        print('Accuracy: %5d %%' % (correct / total * 100))
test()

  0%|          | 0/10000 [00:00<?, ?it/s]

Calculating accuracy...


100%|██████████| 10000/10000 [00:29<00:00, 337.08it/s]

Accuracy:    74 %
